In [6]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# === CONFIG ===
MODEL_PATH = "mobilenetv2_skin_type_model.keras"
TARGET_SIZE = (224, 224)
FALLBACK_CLASS = 'NOT DETECTED RESCAN YOUR FACE'
THRESHOLD = 0.5

# === LOAD MODEL ===
model = load_model(MODEL_PATH)
labels = {0: 'Combination', 1: 'Dry', 2: 'Normal', 3: 'Oily', 4: 'Sensitive'}

# === FACE DETECTOR ===
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# === FUNCTION: Crop and Preprocess Face ===
def preprocess_face(face_img):
    face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    face_img = cv2.resize(face_img, TARGET_SIZE)
    face_array = img_to_array(face_img)
    face_array = preprocess_input(face_array)  # Scale [-1, 1]
    return np.expand_dims(face_array, axis=0)

# === FUNCTION: Predict Skin Type ===
def give_feedback_and_predict(face):
    
    input_data = preprocess_face(face)
    preds = model.predict(input_data, verbose=0)[0]

    print("\n🔍 Softmax Probabilities:")
    for i, prob in enumerate(preds):
        print(f"{labels[i]}: {prob:.4f}")

    top_idx = np.argmax(preds)
    confidence = preds[top_idx]

    if confidence < THRESHOLD:
        final_label = FALLBACK_CLASS
    else:
        final_label = labels[top_idx]

    print(f"\n✅ Predicted Skin Type: {final_label} (Confidence: {confidence:.2f})")
    return final_label, preds

# === FUNCTION: Check Lighting ===
def check_lighting(face_img):
    gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    brightness = np.mean(gray)
    return brightness > 80

# === CAPTURE LOOP ===
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # Use HD for better skin texture
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

print("\n🔵 Press 'c' to capture | Press 'ESC' to exit\n")

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to grab frame.")
        break

    frame_copy = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(120, 120))
    feedback_msg = "No face detected"

    for (x, y, w, h) in faces:
        # Adjust cropping to focus tightly
        margin = 0.1
        x1 = max(int(x + w * margin), 0)
        y1 = max(int(y + h * margin), 0)
        x2 = min(int(x + w * (1 - margin)), frame.shape[1])
        y2 = min(int(y + h * (1 - margin)), frame.shape[0])

        face_crop = frame[y1:y2, x1:x2]

        # Draw rectangle
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Check brightness
        if not check_lighting(face_crop):
            feedback_msg = "⚠️ Increase lighting"
        else:
            feedback_msg = "✅ Face detected - Press 'c' to predict"

        # Show feedback
        cv2.putText(frame, feedback_msg, (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                    (0, 0, 255) if "⚠️" in feedback_msg else (0, 255, 0), 2)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('c') and "✅" in feedback_msg:
            label, _ = give_feedback_and_predict(face_crop)
            cv2.putText(frame, f"Predicted: {label}", (10, frame.shape[0] - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 255), 2)

    cv2.imshow("Skin Type Detection", frame)

    

    if cv2.waitKey(5) & 0xFF == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()



🔵 Press 'c' to capture | Press 'ESC' to exit


🔍 Softmax Probabilities:
Combination: 0.1608
Dry: 0.0009
Normal: 0.0467
Oily: 0.0145
Sensitive: 0.7771

✅ Predicted Skin Type: Sensitive (Confidence: 0.78)


In [4]:
!pip install opencv-python
